# Stock Price Prediction - Notebook 1 - Overview, Loading and Cleaning <a class="tocSkip">

This notebook contains the initial explanation of the project I am looking to deliver and the loading and cleaning of the data.

## Introduction
**Problem Statement**

Stock price prediction is a feat which has been attempted by many individuals and organisations alike involved in the finance and investment space of which many would argue is not possible to achieve at a significant level of accuracy. The purpose of predicting stock prices is for individuals or organisations to gain in profits from investing in them. 

However it is argued that it is very difficult to achieve good results which would lead to profit as there are several factors which instigate volatile and unpredictability in the price movement of a stock. One such factor is irrational behaviour, for example in January 2021, Gamestop shares saw a drastic increase in price which was led by a large increase in trading by retail investors. This was organised via social media, in particular the WallStreetBets chat forum on Reddit (which resulted in the forum being shutdown). Prices were reported to show a 2700% increase by January 27th 2021 in the space of 3 weeks(https://www.ft.com/content/df758a2a-6caf-4d5f-ab70-bb5815922b91). It is these kinds of situations where a stock is unpredictable and would lead to a good workiing model suddenly producing bad results. 

Off course there are other factors such as perceived intrinsic value, macro economics, technical indicators and so on. All of these factors make the prices of many stocks dynamic and volatile which make it difficult to predict prices with a high level of accuracy.

Nonetheless I look to explore whether a stock can be predicted solely using the historical pricing data of the stock and investigating whether we can use these models to predict some stocks better than others. Ultimately building a workflow where anybody can input a stock into the model and compare the performance of the model between different stocks. Thus giving individuals and organisations a view of whether a stock is 'easier' to predict than others and perhaps working towards a more complex model in the future.

**The Data**

The Data that will be used in this project will be fetched from Yahoo Finance through pandas datareader. This data is in the the form of a dataframe and provides historical price data of the specified stock over a specified date range.  Further information can be found at https://pandas-datareader.readthedocs.io/en/latest/readers/yahoo.html. I will be using Apple as my initial stock to model off of and then attempt the model on other stocks and compare performance.

<br>

## Importing Libraries and Data from Yahoo Finance

In my first step I have imported the relevant libraries below.

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data
import joblib

from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns
import math

### Loading Apple Stock Price Data

Below I use DataReader, specifying 'yahoo' as the source to request data from yahoo finance. The parameters for ticker, start date and end date were specified beforehand which inputs the stock and date ranges respectively.

In [2]:
# Define the stock price ticker which we would like to load prices for
ticker = 'AAPL'

# We would like all available data from 2000-01-01 until 2020-12-31
start_date = '2000-01-01'
end_date = '2020-12-31'

# Using pandas data.DataReader to load the desired data inputting the parameters previously specified
apple_df = data.DataReader(ticker, 'yahoo', start_date, end_date)

###  Creating a Function to Load Historical Stock Price Data

I create a helper function to be able to easily import any stock with a known ticker between a specified date range

In [3]:
def import_stock(ticker, start_date, end_date):
    '''This function imports historical stock price data in the form of a dataframe for a specified date range from
    yahoo finance.
    
    ticker: specify the stock ticker as a string (e.g. 'AAPL')
    start_date: specify the start date as a string (e.g. '2000-01-01')
    end_date: specify the end date as a string (e.g. '2020-12-31')\
    '''
    
    return data.DataReader(ticker, 'yahoo', start_date, end_date)

<br>
Below I have tested the helper function to check that it works.

In [4]:
amazon = import_stock('AMZN','2000-01-01','2020-12-31')

In [5]:
amazon

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,4.478125,3.952344,4.075000,4.468750,322352000,4.468750
2000-01-04,4.575000,4.087500,4.268750,4.096875,349748000,4.096875
2000-01-05,3.756250,3.400000,3.525000,3.487500,769148000,3.487500
2000-01-06,3.634375,3.200000,3.565625,3.278125,375040000,3.278125
2000-01-07,3.525000,3.309375,3.350000,3.478125,210108000,3.478125
...,...,...,...,...,...,...
2020-12-24,160.100006,158.449997,159.695007,158.634506,29038000,158.634506
2020-12-28,165.199997,158.634506,159.699997,164.197998,113736000,164.197998
2020-12-29,167.532501,164.061005,165.496994,166.100006,97458000,166.100006


<br>

## Initial Look at the Data and Cleaning

I will begin this process by intiially looking at the dataframe and inspect the null values, data types and dataframe in general

### Shape, Structure and Quality of the Data

In [6]:
# calling apple_df to show head and tail of the dataframe
apple_df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,1.004464,0.907924,0.936384,0.999442,535796800.0,0.851942
2000-01-04,0.987723,0.903460,0.966518,0.915179,512377600.0,0.780115
2000-01-05,0.987165,0.919643,0.926339,0.928571,778321600.0,0.791531
2000-01-06,0.955357,0.848214,0.947545,0.848214,767972800.0,0.723033
2000-01-07,0.901786,0.852679,0.861607,0.888393,460734400.0,0.757282
...,...,...,...,...,...,...
2020-12-24,133.460007,131.100006,131.320007,131.970001,54930100.0,130.404587
2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,135.068604
2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,133.270172


In [7]:
# Calling .info() to check for number of rows, null values and data types
apple_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5284 entries, 2000-01-03 to 2020-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       5284 non-null   float64
 1   Low        5284 non-null   float64
 2   Open       5284 non-null   float64
 3   Close      5284 non-null   float64
 4   Volume     5284 non-null   float64
 5   Adj Close  5284 non-null   float64
dtypes: float64(6)
memory usage: 289.0 KB


In [8]:
# Checking for Duplicated Rows
print(f'There are {apple_df.duplicated().sum()} duplicated rows')

print('')

# Checking for Duplicated Columns
print(f'There are {apple_df.T.duplicated().sum()} duplicated columns')

There are 0 duplicated rows

There are 0 duplicated columns


**The following observations can be seen:**
- There are no null values in the dataframe
- There are no duplicates in the dataframe
- The dataframe has a total of 5289 rows and 6 columns, being of shape: (5289, 6)
- The data types for each column are all correct 
- The index of the dataset is the DateTime of the record for the data. This is what we want for analysis and modelling for a timeseries.

<br>

I can conclude that this dataset is clean as expected although there is one thing that we need to address which is the date period between the pricing data. We know that yahoo provides the data for every day that the stock has traded. We also know that stock exchanges and in this case the New York Stock Exchange does not operate on weekends and bank holidays. 

This suggests that the data will not be in the form of regular intervals of date. In particular weekends aren't so much of a problem as we can base our data on business days, however bank holidays tend to fall on weekdays. I require the periodicity to be regular for modelling and therefore will need to address these missing rows for weekdays.

### Looking at Missing Days and Filling in Values

For modelling I require the frequency between days to be equal. It is important to first check the missing days in the dataset and then set this to business days. This ensure that the data as whole is of regular intervals of date. In this case we are looking at Monday to Friday.

I will be using forward fill to fill in any values for dates that I have added. The reasoning here would be due to the price not having moved on that day. As we know that these days are non-trading days, the prices will have not changed as the markets were not open. Therefore we will just use the previous days value to fill in to represent a stagnant movement.

In [9]:
# Setting parameters for the first and last days in the dataset
first_day = apple_df.index.min()
last_day = apple_df.index.max()

# Creating a new variable which creates an index of all of the days between the first and last day
full_range = pd.date_range(start=first_day, end=last_day, freq="D")

# Finding the difference between the full range of days between the date range and the dates in the dataset
# There are 2385 dates missing from the dataset compared to the full range of dates
full_range.difference(apple_df.index)

DatetimeIndex(['2000-01-08', '2000-01-09', '2000-01-15', '2000-01-16',
               '2000-01-17', '2000-01-22', '2000-01-23', '2000-01-29',
               '2000-01-30', '2000-02-05',
               ...
               '2020-11-29', '2020-12-05', '2020-12-06', '2020-12-12',
               '2020-12-13', '2020-12-19', '2020-12-20', '2020-12-25',
               '2020-12-26', '2020-12-27'],
              dtype='datetime64[ns]', length=2385, freq=None)

In [10]:
# I then set the frequency in the dataset to 'B' which sets the index to include all the business days
apple_df_clean = apple_df.asfreq("B")

In [11]:
# I then check the number of days that have NULL values in the new 'apple_cleaned_df'
# This can also be interpreted as the number of days that have been added to the dataset
apple_df_clean.isna().sum()

High         195
Low          195
Open         195
Close        195
Volume       195
Adj Close    195
dtype: int64

In [12]:
# As rows have been added by additionals days on the index of the dataframe I can just find the indexes of these rows
# by checking any column. In this case I have used 'Adj Close' and set isna to true and called on the index
# I then set this to a new variable so that I can check later after filling these values in
apple_na_index = apple_df_clean[apple_df_clean['Adj Close'].isna() == True].index
apple_na_index

DatetimeIndex(['2000-01-17', '2000-02-21', '2000-04-21', '2000-05-29',
               '2000-07-04', '2000-09-04', '2000-11-23', '2000-12-25',
               '2001-01-01', '2001-01-15',
               ...
               '2019-12-25', '2020-01-01', '2020-01-20', '2020-02-17',
               '2020-04-10', '2020-05-25', '2020-07-03', '2020-09-07',
               '2020-11-26', '2020-12-25'],
              dtype='datetime64[ns]', name='Date', length=195, freq=None)

In [13]:
# From the first check the NULL values can be seen below
apple_df_clean.loc[apple_na_index]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-17,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-21,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-21,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-29,NaN,NaN,NaN,NaN,NaN,NaN
2000-07-04,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2020-05-25,NaN,NaN,NaN,NaN,NaN,NaN
2020-07-03,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# I use fillna with the methood 'ffill' which stands for forward fill. This fills in the NULL values with the previous
# Non-Null value. The new sum of NULL values show 0 NULL values
apple_df_clean.fillna(method = 'ffill', inplace = True)
apple_df_clean.isna().sum()

High         0
Low          0
Open         0
Close        0
Volume       0
Adj Close    0
dtype: int64

In [15]:
# We can see that these values have been filled for these rows
apple_df_clean.loc[apple_na_index]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-17,0.912946,0.887277,0.892857,0.896763,390376000.0,0.764417
2000-02-21,1.030134,0.989955,1.023438,0.993304,233441600.0,0.846710
2000-04-21,1.113839,1.045201,1.104353,1.061384,722120000.0,0.904743
2000-05-29,0.802455,0.761161,0.785714,0.771205,181148800.0,0.657389
2000-07-04,0.969866,0.930804,0.930804,0.952009,70828800.0,0.811510
...,...,...,...,...,...,...
2020-05-25,79.807503,78.837502,78.942497,79.722504,81803200.0,78.499626
2020-07-03,92.617500,90.910004,91.962502,91.027496,114041600.0,89.631210
2020-09-07,123.699997,110.889999,120.070000,120.959999,332607200.0,119.319321


In [16]:
# The final head and tail of the dataset
# The new shape of the dataset is now (5479, 6)
apple_df_clean

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-03,1.004464,0.907924,0.936384,0.999442,535796800.0,0.851942
2000-01-04,0.987723,0.903460,0.966518,0.915179,512377600.0,0.780115
2000-01-05,0.987165,0.919643,0.926339,0.928571,778321600.0,0.791531
2000-01-06,0.955357,0.848214,0.947545,0.848214,767972800.0,0.723033
2000-01-07,0.901786,0.852679,0.861607,0.888393,460734400.0,0.757282
...,...,...,...,...,...,...
2020-12-25,133.460007,131.100006,131.320007,131.970001,54930100.0,130.404587
2020-12-28,137.339996,133.509995,133.990005,136.690002,124486200.0,135.068604
2020-12-29,138.789993,134.339996,138.050003,134.869995,121047300.0,133.270172


### The Columns

Below I explain what each column represents and what I will be focusing on for my modelling

 - High: The highest price reached for the specified day
 - Low: The lowest price for the specified day
 - Open: The price of the stock when the market opened for the specified day
 - Close: The price of the stock when the market closed on the specified day
 - Volume: The total nnumber of shares traded on the specified day
 - Adj Close: The adjusted price of the stock when the market closed, taking into account corporate actions such as  stock splits, dividends, and rights offerings
 
For now I will keep all of the columns, However it is important to note that my target variable is the Adj Close price. The adjusted close price is a more accurate representation of the stocks actual closing price compared to close price. This is because corporate actions affect the supply and demand of the stock and therefore this should be taken into account. 

I will be looking to predict this value by looking at the historical price of the dependent variable itself (Adj Close) in my models. Thus using models. which leverage the lags and moving averages of the Adj Close Price.

## Saving the Cleaned Data

No that I have a clean dataset I will proceed with saving this. By saving the data as a pkl file I can load this in my next notebook to conduct EDA. I will also continue saving these files for further notebooks within this project.

In [17]:
# Saving cleaned data as a pickle file using joblib
joblib.dump(apple_df_clean, 'apple_df_clean.pkl')

['apple_df_clean.pkl']